In [113]:
import pandas
import numpy as np
import urllib
import googlemaps
import json
import time, datetime
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
from string import punctuation
from bs4 import BeautifulSoup
import math
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
import random
from sklearn.metrics  import log_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cluster import KMeans

In [2]:
# panda_data = pandas.read_json('../train.json')
# panda_test_data = pandas.read_json('test.json')

In [3]:
# with open('records.json', 'w') as outfile:
#     json.dump(json.loads(panda_data.to_json(orient='records')), outfile)
    
# with open('records_test.json', 'w') as outfile:
#     json.dump(json.loads(panda_test_data.to_json(orient='records')), outfile)

In [3]:
def expressStatus(status):
    print status
    os.system('say "' + str(status) + '"')
    

In [15]:
# def parseData(fname):
#     for l in urllib.urlopen(fname):
#         yield eval(l)
        
# data = np.transpose(list(parseData('records.json')))
# test_data = np.transpose(list(parseData('records_test.json')))

In [6]:
for i, k in enumerate(data[0][0]):
    print i+1, '.', k, ':', data[0][0][k]


1 . display_address : Metropolitan Avenue
2 . description : A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To L Train<br /><br />Don't miss out!<br /><br />We have several great apartments in the immediate area.<br /><br />For additional information 687-878-2229<p><a  website_redacted 
3 . price : 3000
4 . bedrooms : 3
5 . photos : ['https://photos.renthop.com/2/7211212_1ed4542ec81621d70d1061aa833e669c.jpg', 'https://photos.renthop.com/2/7211212_7dfc41dced69245065df83d08eed4a00.jpg', 'https://photos.renthop.com/2/7211212_c17853c4b869af6f53af08b0f5820b4c.jpg', 'https://photos.renthop.com/2/7211212_787ad8ea0c089792e7453e2121f8ac89.j

In [5]:
# def stem_tokens(tokens, stemmer):
#     stemmed = []
#     for item in tokens:
#         stemmed.append(stemmer.stem(item))
#     return stemmed

# def tokenize(text, stemmer=PorterStemmer()):
#     tokens = nltk.word_tokenize(text)
#     stems = stem_tokens(tokens, stemmer)
#     return stems

# tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

# def trainTfIdfModel(strings):
#     content = np.array([])
#     tfIdfScores = np.array([])
#     print 'Preprocessing Data'
#     for s in strings:
#         t = " ".join(item.strip() for item in BeautifulSoup(s).find_all(text=True))
#         t = str(''.join([i if ord(i) < 128 else ' ' for i in t]))
#         replace_punctuation = string.maketrans(punctuation, ' '*len(punctuation))
#         transformed_string = t.lower().translate(replace_punctuation)
#         content = np.append(content, transformed_string)
     
#     print 'Started Training'
#     tfs = tfidf.fit_transform(content)

# trainTfIdfModel([d[0]['description'] for d in data])

# def getTransformedtfIdfScore(review):
#     t = " ".join(item.strip() for item in BeautifulSoup(review).find_all(text=True))
#     t = str(''.join([i if ord(i) < 128 else ' ' for i in t]))
#     replace_punctuation = string.maketrans(punctuation, ' '*len(punctuation))
#     transformed_string = t.lower().translate(replace_punctuation)
#     response = tfidf.transform([transformed_string])
    
# #     feature_names = tfidf.get_feature_names()
# #     for col in response.nonzero()[1]:
# #         print feature_names[col], ' - ', response[0, col]
    
# #     print response.nonzero()[1]
#     if float(response.nonzero()[1].shape[0]) == 0:
#         return 0
#     return float(np.sum([response[0, col] for col in response.nonzero()[1]])) / float(response.nonzero()[1].shape[0])

In [16]:
# def getARI(string):
#     string = " ".join(item.strip() for item in BeautifulSoup(string).find_all(text=True))
#     string = str(''.join([i if ord(i) < 128 else ' ' for i in string]))
#     s_count = float(len(nltk.sent_tokenize(string)))
#     w_count = float(len(nltk.wordpunct_tokenize(string)))
#     c_count = float(len(string))
    
#     if s_count == 0:
#         return 12
    
#     return math.ceil((4.71*(c_count/w_count)) + (0.5*(w_count/s_count)) - 21.43)



# def GetFeatures(data):
#     structured_data = []
#     index = 1
#     for d in data:
#         feat = []
#         if index % 10000 == 1:
#             print 'Starting', index
            
#         feat.append(time.mktime(datetime.datetime.strptime(d[0]['created'], "%Y-%m-%d %H:%M:%S").timetuple()))
#         feat.append(getTransformedtfIdfScore(d[0]['description']))
#         feat.append(getARI(d[0]['description']))
#         structured_data.append(feat)
#         if index % 10000 == 1:
#             print 'Done', index
#         index += 1
    
#     return structured_data

In [17]:
# new_data = GetFeatures(data)
# new_test_data = GetFeatures(test_data)

Starting 1
Done 1
Starting 10001
Done 10001
Starting 20001
Done 20001
Starting 30001
Done 30001
Starting 40001
Done 40001
Starting 1
Done 1
Starting 10001
Done 10001
Starting 20001
Done 20001
Starting 30001
Done 30001
Starting 40001
Done 40001
Starting 50001
Done 50001
Starting 60001
Done 60001
Starting 70001
Done 70001


In [19]:
print new_data[0]
print np.array(new_data).shape

print new_test_data[0]
print np.array(new_test_data).shape

 [1466780064.0, 0.10791517320475999, 6.0]
(49352, 3)
[1465648181.0, 0.12883028455155554, 29.0]
(74659, 3)


In [20]:
np.save('ut_tfidf_ari_train', new_data)
np.save('ut_tfidf_ari_test', new_test_data)

In [114]:
feature_names = []

feature_names.append('bathrooms'),#0
feature_names.append('bedrooms'),#1
feature_names.append('density'),#2
feature_names.append('interest_manager_low'),#3
feature_names.append('interest_manager_medium'),#4
feature_names.append('interest_manager_high'),#5
feature_names.append('interest_manager_count'),#6
feature_names.append('interest_building_low'),#7
feature_names.append('interest_building_medium'),#8
feature_names.append('interest_building_high'),#9
feature_names.append('interest_building_count'),#10
feature_names.append('number of features'),#11
feature_names.append('latitude'),#12
feature_names.append('longitude'),#13
feature_names.append('distance from city center'),#14
feature_names.append('number of photos'),#15
feature_names.append('population'),#16
feature_names.append('price'),#17
feature_names.append('wage'),#18
feature_names.append('unix timestamp'),#19
feature_names.append('tfidf of description'),#20
feature_names.append('ari of desciption'),#21,
feature_names.append('interest level')#22
feature_names.append('building id'),#21,
feature_names.append('manager id')#22
feature_names.append('#bedrooms / (#bedrooms + #bathrooms + 1)')#21,
feature_names.append('KMeans-LatLong')#21,
                                                 
np.save('feature_names', feature_names)
all_features = np.load('feature_names.npy')
train_feature_index = dict([(v,i) for i,v in enumerate(all_features)])
for feature in train_feature_index:
    print feature, train_feature_index[feature]

KMeans-LatLong 26
bathrooms 0
interest_building_high 9
interest_manager_count 6
distance from city center 14
interest_building_low 7
wage 18
ari of desciption 21
density 2
interest level 22
number of features 11
latitude 12
interest_building_medium 8
unix timestamp 19
price 17
bedrooms 1
building id 23
interest_building_count 10
interest_manager_high 5
population 16
#bedrooms / (#bedrooms + #bathrooms + 1) 25
tfidf of description 20
interest_manager_medium 4
longitude 13
manager id 24
number of photos 15
interest_manager_low 3


In [115]:
print all_features.shape

(27,)


In [116]:
feature_names_test = []

feature_names_test.append('bathrooms'),#0
feature_names_test.append('bedrooms'),#1
feature_names_test.append('density'),#2
feature_names_test.append('interest_manager_low'),#3
feature_names_test.append('interest_manager_medium'),#4
feature_names_test.append('interest_manager_high'),#5
feature_names_test.append('interest_manager_count'),#6
feature_names_test.append('interest_building_low'),#7
feature_names_test.append('interest_building_medium'),#8
feature_names_test.append('interest_building_high'),#9
feature_names_test.append('interest_building_count'),#10
feature_names_test.append('number of features'),#11
feature_names_test.append('latitude'),#12
feature_names_test.append('longitude'),#13
feature_names_test.append('distance from city center'),#14
feature_names_test.append('number of photos'),#15
feature_names_test.append('population'),#16
feature_names_test.append('price'),#17
feature_names_test.append('wage')#18
feature_names_test.append('unix timestamp'),#19
feature_names_test.append('tfidf of description'),#20
feature_names_test.append('ari of desciption')#21,
feature_names_test.append('building id'),#20
feature_names_test.append('manager id')#21,
feature_names_test.append('#bedrooms / (#bedrooms + #bathrooms + 1)')#21,
feature_names_test.append('KMeans-LatLong')#21,

np.save('feature_names_test', feature_names_test)

all_features_test = np.load('feature_names_test.npy')
test_feature_index = dict([(v,i) for i,v in enumerate(all_features_test)])
for feature in test_feature_index:
    print feature, test_feature_index[feature]

KMeans-LatLong 25
bathrooms 0
interest_building_high 9
interest_manager_count 6
distance from city center 14
interest_building_low 7
wage 18
ari of desciption 21
density 2
number of features 11
latitude 12
interest_building_medium 8
unix timestamp 19
price 17
bedrooms 1
building id 22
interest_building_count 10
interest_manager_high 5
population 16
#bedrooms / (#bedrooms + #bathrooms + 1) 24
tfidf of description 20
interest_manager_medium 4
longitude 13
manager id 23
number of photos 15
interest_manager_low 3


In [138]:
chosen_feature_indices = dict()
def extract_features(datum, index_dictionary, first_building_review_time, first_manager_review_time):
    feat = []
    index = 0
#     feat.append(datum[index_dictionary['bathrooms']])
#     chosen_feature_indices[index] = 'bathrooms'
#     index += 1
    
#     feat.append(datum[index_dictionary['bedrooms']])
#     chosen_feature_indices[index] = 'bedrooms'
#     index += 1
    
    feat.append(datum[index_dictionary['interest_manager_count']])
    chosen_feature_indices[index] = 'interest_manager_count'
    index += 1
    
    feat.append(datum[index_dictionary['interest_building_count']])
    chosen_feature_indices[index] = 'interest_building_count'
    index += 1
    
    feat.append(datum[index_dictionary['distance from city center']])
    chosen_feature_indices[index] = 'distance from city center'
    index += 1
    
    feat.append(datum[index_dictionary['wage']])
    chosen_feature_indices[index] = 'wage'
    index += 1
    
    feat.append(datum[index_dictionary['price']])
    chosen_feature_indices[index] = 'price'
    index += 1
    
    feat.append(datum[index_dictionary['ari of desciption']])
    chosen_feature_indices[index] = 'ari of desciption'
    index += 1
    
    feat.append(datum[index_dictionary['density']])
    chosen_feature_indices[index] = 'density'
    index += 1
    
    feat.append(datum[index_dictionary['number of features']])
    chosen_feature_indices[index] = 'number of features'
    index += 1
    
    feat.append(datum[index_dictionary['unix timestamp']])
    chosen_feature_indices[index] = 'unix timestamp'
    index += 1
    
    feat.append(datum[index_dictionary['population']])
    chosen_feature_indices[index] = 'population'
    index += 1
    
    feat.append(datum[index_dictionary['tfidf of description']])
    chosen_feature_indices[index] = 'tfidf of description'
    index += 1
    
    feat.append(datum[index_dictionary['number of photos']])
    chosen_feature_indices[index] = 'number of photos'
    index += 1
    
    
#     feat.append(datum[index_dictionary['latitude']])
#     chosen_feature_indices[index] = 'latitude'
#     index += 1
    
#     feat.append(datum[index_dictionary['longitude']])
#     chosen_feature_indices[index] = 'longitude'
#     index += 1
    
    feat.append(float(datum[index_dictionary['bedrooms']])/(1 + datum[index_dictionary['bedrooms']] + datum[index_dictionary['bathrooms']]))
    chosen_feature_indices[index] = '#bedrooms / (#bedrooms + #bathrooms + 1)'
    index += 1
    
    if first_building_review_time.get(datum[index_dictionary['building id']], -1) == -1:
        feat.append(datum[index_dictionary['unix timestamp']])
    else:
        feat.append(first_building_review_time[datum[index_dictionary['building id']]])
    chosen_feature_indices[index] = 'first building review time'
    index += 1
    
    if first_manager_review_time.get(datum[index_dictionary['manager id']], -1) == -1:
        feat.append(datum[index_dictionary['unix timestamp']])
    else:
        feat.append(first_manager_review_time[datum[index_dictionary['manager id']]])
    chosen_feature_indices[index] = 'first manager review time'
    index += 1
    
    feat.append(float(datum[index_dictionary['price']])/(datum[index_dictionary['bedrooms']] + 1))
    chosen_feature_indices[index] = 'average bedroom price'
    index += 1
    
    feat.append(float(datum[index_dictionary['price']])/(datum[index_dictionary['bathrooms']] + 1))
    chosen_feature_indices[index] = 'average bathroom price'
    index += 1
    
    feat.append(float(datum[index_dictionary['price']])/(datum[index_dictionary['bathrooms']] + datum[index_dictionary['bedrooms']] + 1))
    chosen_feature_indices[index] = 'average (bathroom + bedrooms) price'
    index += 1

    
#     feat.append(2*datum[index_dictionary['interest_building_high']] +
#                 1*datum[index_dictionary['interest_building_medium']] +
#                 0*datum[index_dictionary['interest_building_low']])
#     chosen_feature_indices[index] = 'building expectation'
#     index += 1
    
#     feat.append(2*datum[index_dictionary['interest_manager_high']] +
#                 1*datum[index_dictionary['interest_manager_medium']] +
#                 0*datum[index_dictionary['interest_manager_low']])
#     chosen_feature_indices[index] = 'manager expectation'
#     index += 1
    
    return feat

In [139]:
def truncate_data(data, index_dictionary):
    data = np.array([d for d in data if d[index_dictionary['distance from city center']] < 0.2])
    return data


In [140]:
data_features = np.load('featureX.npy')
first_building_review_time = dict()
first_manager_review_time = dict()
unique_buildings = np.unique(data_features[:,train_feature_index['building id']])
unique_managers = np.unique(data_features[:,train_feature_index['manager id']])

for building in unique_buildings:
    first_building_review_time[building] = np.min([d[train_feature_index['unix timestamp']] for d in data_features if d[train_feature_index['building id']] == building]) 

for manager in unique_managers:
    first_manager_review_time[manager] = np.min([d[train_feature_index['unix timestamp']] for d in data_features if d[train_feature_index['manager id']] == manager]) 

    
print data_features.shape, all_features.shape
random.shuffle(data_features)
validation_count = int(data_features.shape[0] * 0.2)
train_data = truncate_data(data_features[:-validation_count], train_feature_index)
train_X = np.array([extract_features(d, train_feature_index, first_building_review_time, first_manager_review_time) for d in train_data])
train_y = np.array([d[train_feature_index['interest level']] for d in train_data])


validation_X = np.array([extract_features(d, train_feature_index, first_building_review_time, first_manager_review_time) for d in data_features[-validation_count:]])
validation_y = np.array([d[train_feature_index['interest level']] for d in data_features[-validation_count:]])

test_features = np.load('X_test.npy')
print test_features.shape, all_features_test.shape
test_X = np.array([extract_features(d, test_feature_index, first_building_review_time, first_manager_review_time) for d in test_features])
print train_X.shape, test_X.shape, train_y.shape, validation_X.shape, validation_y.shape

(49352, 44) (27,)
(74659, 44) (26,)
(39360, 20) (74659, 20) (39360,) (9870, 20) (9870,)


In [141]:
n_estimators = 200
max_features = None
max_depth = 17
oob_score = True
verbose = 2
min_samples_leaf = 17
n_jobs = -1

expressStatus("Training random forest model with " + str(n_estimators) + " estimators and max depth "+ str(max_depth) + " and " + str(train_X[0].shape[0]) + " features")
rf = RandomForestClassifier(max_features=max_features, n_estimators=n_estimators, criterion='entropy', max_depth = max_depth, verbose = verbose, n_jobs=n_jobs, min_samples_leaf=min_samples_leaf, oob_score=oob_score)#, max_features=max_features
rf.fit(train_X, train_y)

train_predictions = rf.predict_proba(train_X)
validation_predictions = rf.predict_proba(validation_X)


print train_predictions.shape, train_predictions[0]
expressStatus("Done training model!")


Training random forest model with 200 estimators and max depth 17 and 20 features
building tree 1 of 200
building tree 2 of 200building tree 3 of 200building tree 4 of 200


building tree 5 of 200
building tree 6 of 200
 building tree 7 of 200
building tree 8 of 200
building tree 9 of 200building tree 10 of 200

building tree 11 of 200building tree 12 of 200

building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200building tree 32 of 200

building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.2s


building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69 of 200
building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77 of 200
building tree 78

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   17.1s



building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200
building tree 187 of 200
building tree 188 of 200
building tree 189 of 200
building tree 190 of 200
building tree 191 of 200
building tree 192 of 200
building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200
building tree 197 of 200
building tree 198 of 200
building tree 199 of 200
building tree 200 of 200

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   22.1s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.3s finished


(39360, 3) [ 0.24562859  0.72712906  0.02724234]
Done training model!


In [142]:
for i, value in sorted(enumerate(rf.feature_importances_), key=lambda x:abs(x[1]), reverse=True):
    print chosen_feature_indices[i], value

average (bathroom + bedrooms) price 0.147013247649
interest_building_count 0.115802185463
price 0.0862177776407
distance from city center 0.0837025081331
tfidf of description 0.0690441072147
unix timestamp 0.063756395202
interest_manager_count 0.0621742721529
average bathroom price 0.058637142865
average bedroom price 0.0564781347019
number of features 0.0486560730227
number of photos 0.0438053278205
ari of desciption 0.0424935287857
wage 0.0336221486483
density 0.0276698557733
population 0.0236550401346
first building review time 0.0184433142849
first manager review time 0.0118807343042
#bedrooms / (#bedrooms + #bathrooms + 1) 0.0029419975826
bathrooms 0.00223652538794
bedrooms 0.00176968323253


In [143]:
print 'Cross entropy loss on train data:', log_loss(train_y, train_predictions)
print 'Cross entropy loss on validation data:', log_loss(validation_y, validation_predictions)

Cross entropy loss on train data: 0.363281258708
Cross entropy loss on validation data: 0.520827622083


In [24]:
test_predictions = np.array(rf.predict_proba(test_X))

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    1.1s finished


In [26]:
output = open('result.txt', 'w')
output.write("listing_id,high,medium,low\n")
for i in range(len(test_X)):
    output.write(str(int(test_features[i][22])) + ',' + str(test_predictions[i][2]) + ',' + str(test_predictions[i][1]) + ',' + str(test_predictions[i][0]) + '\n')
output.close()

print test_predictions[:10]

[[  7.33123290e-01   2.10948313e-01   5.59283969e-02]
 [  8.03923578e-01   1.78740231e-01   1.73361904e-02]
 [  8.83267976e-01   1.14743352e-01   1.98867161e-03]
 [  4.86324419e-01   4.23339492e-01   9.03360894e-02]
 [  8.55014039e-01   1.30915277e-01   1.40706833e-02]
 [  9.65918228e-01   3.35048490e-02   5.76923077e-04]
 [  5.18450979e-01   4.59734572e-01   2.18144487e-02]
 [  8.69000419e-01   1.02068567e-01   2.89310137e-02]
 [  6.42005663e-01   2.89877109e-01   6.81172275e-02]
 [  7.48751010e-01   2.33689997e-01   1.75589927e-02]]


In [65]:
for name in test_feature_index:
    print name, test_features[0][test_feature_index[name]]

bathrooms 1.0
interest_building_high 0.0
interest_manager_count 24.0
distance from city center 0.0690372906913
interest_building_low 0.0
wage 17123.723916
ari of desciption 29.0
density 92511.3636364
number of features 6.0
latitude 40.7185
interest_building_medium 1.0
unix timestamp 1465648181.0
price 2950.0
bedrooms 1.0
interest_building_count 3.0
interest_manager_high 0.208333333333
population 81410.0
tfidf of description 0.128830284552
interest_manager_medium 0.333333333333
longitude -73.9865
number of photos 8.0
interest_manager_low 0.458333333333
